In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',200)
pd.set_option('display.max_rows', 200)

In [ ]:
#readind dataset
df=pd.read_csv("loan.csv",low_memory=False)

In [ ]:
#viewing dataset
df.head(15)

In [ ]:
#shape
df.shape

### Data Cleansing

In [ ]:
#missing or null values in dataset
null_count=(df.isnull().sum())
null_count

In [ ]:
# finding null %
null_perc=(100*null_count/len(df.index))
# rounding value
null_perc=round(null_perc,2)
null_perc

In [ ]:
#Printing values having more than 0% misisng values
null_perc[null_perc>0]

In [ ]:
#removing col having >30% null values
df.drop(null_perc[null_perc>30].index, axis=1, inplace=True)

In [ ]:
#viewing unique values 
df.nunique().sort_values()

we can drop the column values with 1 unique values

In [ ]:
#dropping col with one unique values
df.drop(['tax_liens','delinq_amnt','chargeoff_within_12_mths','acc_now_delinq','application_type','policy_code','collections_12_mths_ex_med','initial_list_status','pymnt_plan'], axis=1,inplace=True)

In [ ]:
#printing unique value
df.nunique().sort_values(ascending=False)

In [ ]:
#taking high unique values 
df.nunique().sort_values(ascending=False).head(15)

In [ ]:
df.dtypes

Dropping some text values which have text values in data set

In [ ]:
df.drop(['emp_title','title'],axis=1,inplace=True)

Dropping url and member id

In [ ]:
df.drop(['url','member_id'],axis=1,inplace=True)

dropping --->out_prncp,out_prncp_inv,out_prncp_inv,total_rec_prncp,total_rec_late_fee,
recoveries,collection_recovery_fee,last_credit_pull_d,last_pymnt_d,
out_prncp,out_prncp_inv 

In [ ]:
df.drop(['total_rec_int', 'total_rec_prncp', 'total_rec_late_fee', 'last_credit_pull_d', 'out_prncp',  'out_prncp_inv', 'last_pymnt_d'], axis=1, inplace=True)

since we are only interested in loan dropping recoveries & collection_recovery_fee

In [ ]:
df.drop(['collection_recovery_fee','recoveries'],axis=1,inplace=True)

In [ ]:
#% of null or misisng values
null_perc = round(100*(df.isnull().sum()/len(df.index)),2)
null_perc[null_perc>0]

In [ ]:
#removing null value rows emp_length,revol_util,pub_rec_bankruptcies
df.dropna(subset=['emp_length','revol_util','pub_rec_bankruptcies'],inplace=True)

In [ ]:
null_perc = round(100*(df.isnull().sum()/len(df.index)),2)
null_perc[null_perc>0]

In [ ]:
df['emp_length'] = df['emp_length'].str.rstrip('years ')
df['emp_length'][df['emp_length'] == '< 1'] = '0.5'
df['emp_length'].replace({'10+': '10'}, inplace=True)

removing current values in loan status bcs w need only fully paid and charged off

In [ ]:
df=df[df.loan_status != 'Current']

In [ ]:
#checking unique values from term column
df.term.unique()

In [ ]:
df['term'] = df.term.str.strip()
df.term.unique()

In [ ]:
df.shape

### Converting Data Type 

In [ ]:
df.dtypes

int_rate and revol_util are having % symbol  lets remove it 

In [ ]:
#removing % values
#f['int_rate'] = df.int_rate.replace('%','').astype(float)
#f['revol_util'] = df.int_rate.replace('%','').astype(float)

df['int_rate'] = df.int_rate.replace('%','', regex=True).astype(float)
df['revol_util'] = df.int_rate.replace('%','',regex=True).astype(float)

converting data-type to dates

In [ ]:
df['issue_d']=pd.to_datetime(df.issue_d,format="%b-%y")
df['issue_d'] = df['issue_d'].apply(lambda x: x-pd.DateOffset(years=100) if x.year > 2021 else x)

In [ ]:
#df['issue_d'].value_counts().sort_index().plot(kind='bar')


In [ ]:
df['earliest_cr_line'] = pd.to_datetime(df.earliest_cr_line, format='%b-%y')
df['earliest_cr_line'] = df['earliest_cr_line'].apply(lambda x: x-pd.DateOffset(years=100) if x.year > 2020 else x)

In [ ]:
#df['earliest_cr_line'].value_counts().sort_index().plot(kind='bar')

In [ ]:
df.dtypes

In [ ]:
#Seperating date-month-year as derived variables

In [ ]:
#deriving week
df['issued_d_year']=df.issue_d.dt.year
#deriving month
df['issued_d_month']=df.issue_d.dt.strftime('%b')
#deriving day
df['issue_d_day']=df.issue_d.dt.weekday

df['earliest_cr_line_year']=df.issue_d.dt.year
df['earliest_cr_line_month']=df.issue_d.dt.strftime('%b')
df['earliest_cr_line_day']=df.issue_d.dt.weekday

In [ ]:
#converting date as object data type
df['issued_d_year']=df['issued_d_year'].astype(object)
df['issued_d_month']=df['issued_d_month'].astype(object)
df['issue_d_day']=df['issue_d_day'].astype(object)
df['earliest_cr_line_year']=df['earliest_cr_line_year'].astype(object)
df['earliest_cr_line_month']=df['earliest_cr_line_month'].astype(object)
df['earliest_cr_line_day']=df['earliest_cr_line_day'].astype(object)

In [ ]:
df.head()

Adding approved loan 

Approved loan = (funded_amount_by+invester*100/ loan_amount)

In [ ]:
df['approved_loan_ratio']= (df.funded_amnt_inv*100)/(df.loan_amnt)

In [ ]:
df['approved_loan_ratio'].head(25)

In [ ]:
df['approved_loan_ratio']=round(df['approved_loan_ratio'],2)

In [ ]:
#df['approved_loan_ratio']

In [ ]:
df.dtypes

Data cleansing part is done, now lets do univariate and multivariate analysis

# 1.Univariate Analysis

Loan Amount

In [ ]:
plt.figure(figsize=(20,10))
sns.countplot(x=df['loan_amnt'],data=df)
plt.show()

Loan Status

In [ ]:
sns.countplot(x=df['loan_status'])
plt.show()

Grade

In [ ]:
sns.countplot(x=df['grade'],data=df)
plt.show()

Employee working experience

In [ ]:
sns.countplot(x=df['emp_length'],data=df)
plt.xlabel('Working Experience')
plt.show()

Funded Amount by Investor

In [ ]:
sns.distplot(df['funded_amnt_inv'])
plt.show()

In [ ]:
df['funded_amnt_inv'].describe()

Loan amount

In [ ]:
sns.distplot(df['loan_amnt'])
plt.show()

Annual Income of the employee

In [ ]:
#reviwing salary

sns.boxplot(x=df['annual_inc'])
plt.show()

Approved Loan Ratio

In [ ]:
sns.distplot(a=df['approved_loan_ratio'])
plt.show()

## Observations-Univariate

1.It is observed that the loan amount varies from 0 to 30,000

2.Mean = 10,000

3.Most of the loans are fully paid

4.Most of the loans are grade A & B

5.Human having more than 10 years experience apply loan in majority

6.Funded amount by investor is right skewed, hence most of the loan amount is greater than 3.5Lakhs

7.From the above plot it is visible that only 2 Humans are getting higher salary greater than 3L

8.70% of people get sure loan from investers

## 2.Multivariate Analysis

Employee Exp vs Loan Status

In [ ]:
plt.figure(figsize=(15,15))
sns.countplot(x=df['emp_length'], hue=df['loan_status'], data=df)
plt.xlabel('Employee Working Experience')
plt.show()

Loan Amount vs Funded amount by Investors

In [ ]:
plt.figure(figsize=(15,15))
sns.barplot(x=df['funded_amnt_inv'],y=df['loan_amnt'],data=df)
plt.show()

Loan Amount vs Loan Status

In [ ]:
plt.figure(figsize=(15,15))
sns.boxplot( y=df["loan_amnt"], x=df['loan_status'] , data=df)
plt.ylabel('Loan Amount')
plt.show()

LoanPurpose vs LoanStatus

In [ ]:
plt.figure(figsize=(15,15))
sns.countplot(x=df['purpose'], hue=df['loan_status'], data=df)
plt.title('Loan Reason')
plt.xticks(rotation=90)
plt.show()

Term vs Purpose

In [ ]:
plt.figure(figsize=(15,8))
sns.countplot( x=df['term'], hue=df['purpose'], data=df)
plt.xlabel('Loan Term in Months')
plt.show()

Term vs Loan Amount

In [ ]:
plt.figure(figsize=(15,15))
sns.barplot(x=df['term'],y=df['loan_amnt'],hue=df['loan_status'])
plt.show()

Term vs Interest

In [ ]:
plt.figure(figsize=(15,15))
sns.barplot(x=df['term'],y=df['int_rate'],hue=df['loan_status'])

Interest Rate vs Annual Income

In [ ]:
plt.figure(figsize=(15,15))
sns.barplot(x=df['int_rate'],y=df['annual_inc'])

Loan Amount vs Purpose

In [ ]:
plt.figure(figsize=(10,10))
sns.barplot(x=df['loan_amnt'],y=df['purpose'],hue=df['loan_status'])

Grade vs Loan Amount

In [ ]:
plt.figure(figsize=(15,15))
sns.barplot(x=df['grade'],y=df['loan_amnt'],hue=df['loan_status'])
plt.show()

Grade vs Installment Rate

In [ ]:
plt.figure(figsize=(15,15))
sns.barplot(x=df['grade'],y=df['int_rate'])

In [ ]:
plt.figure(figsize=(15,15))
sns.barplot(x=df['grade'], y=df['int_rate'], hue ='loan_status',data =df)
plt.show()

Loan Amount vs HouseOwnerShip

In [ ]:
plt.figure(figsize=(15,15))
sns.barplot(x=df['home_ownership'], y=df['loan_amnt'],hue=df['loan_status'])
plt.show()

HouseOwnerShip vs AnnualIncome

In [ ]:
plt.figure(figsize=(15,15))
sns.barplot(x=df['home_ownership'],y=df['annual_inc'])
plt.show()

HouseOwnerShip vs Approved loan 

In [ ]:
plt.figure(figsize=(15,15))
sns.boxplot(x=df['home_ownership'],y=df['approved_loan_ratio'])
plt.show()

LoanAmount vs TotalPayment

In [ ]:
plt.figure(figsize=(15,15))
sns.lineplot(x=df['loan_amnt'], y=df['total_pymnt'],data =df)
plt.show()

LoanAmount vs IssuedMonth

In [ ]:
plt.figure(figsize=(15,15))
sns.barplot(x=df['issued_d_month'],y=df['loan_amnt'],hue=df['loan_status'])
plt.show()

IssuedYear vs LoanAmount

In [ ]:
plt.figure(figsize=(15,15))
sns.barplot(x=df['issued_d_year'],y=df['loan_amnt'],hue=df['loan_status'])
plt.show()

## Observations-Multivariate

1.Human's having experince of 10 paid their loan sucessfully when comapred with others,where as 8,9 are not paying theri loan well

2.From the above plot it is observed that mean, 25% and 75% Loan amount of Fully paid and charged off is similiar

3.Major human takes loan for the dept consolidation and taken loan is fully paid

4.Human choosing 36 Months tenure have high chance of paying correctly

5.People having Good annual income paying better Installments

6.Maximum stauts of the loan were Fully paid 

7.People having Home ownership as 'MORTAGE' have income of about 60K

8.F grade Loan applier having loan amount of 15k-20k

9.G grade Loan applier having interest rate above 20%

10.Its seems that the default rate is high in 60 months tenure

11.The Lower grade are higher chance of default than the Higher grade one

12.The lowe grade (E-G) are getting loans for higher interest rate which might cause default

13.Human who took loan for small business seems to be default more 

14.Human's in mortage home ownership has high default rate